# SST Individual Discriminability

This notebook measures discriminability of conditions for each subject, within runs.

It then takes the average discriminability across any multiple runs (I don't think we have them though).

Then we can compare discriminability against other subject-level variables.



Something similar was previously been done in `rsa_within_subj.ipynb`; it took similarity between matrices representing the averages of GNG and compared them intra-class similarity.

That file doesn't take the next step of trying to see how that similarity score might measure up against across-subject variables; it seems, because I did not find good evidence that intrasimilarity was actually higher than interclass similarity.

Conclusions:

 - maybe I should have tried discriminability, not just similarity
 - If we're interested in discriminability, then running an ML algorithm should be superior?
 - But don't rule out going back to similarity.

### General set-up

## TESQ

### TESQ Machine learning set-up

Some of the below can probably be moved to "general set-up" as appropriate.

In [2]:
from discriminability_tools import *

python initialized for apply_loocv_and_save


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


28 25
28


In [3]:
import pickle
from IPython.core.display import display, HTML, Markdown
from nilearn.decoding import Decoder
from sklearn.model_selection import StratifiedKFold
from random import randint
import math

In [4]:
import sys
import os
import pandas as pd
import gc

sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *
from dev_wtp_io_utils import *
import gc
import nibabel as nib

from os import path



nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
train_test_markers_filepath = ml_data_folderpath + "/train_test_markers_20210601T183243.csv"
test_train_df = pd.read_csv(train_test_markers_filepath)

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")


dataset_name = 'conditions'

from nilearn.decoding import DecoderRegressor, Decoder

script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'
# HRF 2s

#get a PFC mask
#pfc_mask = create_mask_from_images(get_pfc_image_filepaths(ml_data_folderpath + "/"),threshold=10)


def trialtype_resp_trans_func(X):
    return(X.trial_type)



In [5]:

cpus_available = int(os.getenv('CPUS_PER_TASK'))
#custom thing I have set in my jupyter notebook task.
print(cpus_available)
cpus_to_use = cpus_available-1

4


### TESQ Analysis set-up

In [6]:
from analyze_results import remove_selected_outliers
from scipy.stats import pearsonr,spearmanr
from matplotlib import pyplot

def remove_selected_outliers_tesq_study(ind_div_combined,show_plot=False):
    idc_outliers_removed = remove_selected_outliers(ind_div_combined,
    ['discriminability_overfit_accuracy','discriminability_mean_cv_scores',
        'BFI_extraversion','RMQ_locomotion','ses_aggregate','PLAN_cognitive_strategies',
     'SST_SSRT','BIS_11','BSCS','TESQ_E_suppression', 'TESQ_E_avoidance_of_temptations', 
     'TESQ_E_goal_deliberation', 'TESQ_E_controlling_temptations', 'TESQ_E_distraction',
     'TESQ_E_goal_and_rule_setting','EDM','RS','TRSQ','ROC_Crave_Regulate_Minus_Look',
     'SRHI_unhealthy',
     'cancer_promoting_minus_preventing_FFQ','bf_1'],
    show_plot=show_plot)
    return(idc_outliers_removed)

def display_discriminability_correlations(ind_div_combined_3sd):
    for neural_var in ['discriminability_overfit_accuracy','discriminability_mean_cv_scores']:
        display(Markdown("### " + neural_var))
        for correlate in ['BFI_extraversion','RMQ_locomotion','ses_aggregate','PLAN_cognitive_strategies',
                          'SST_SSRT','BIS_11','BSCS',
                          'TESQ_E_suppression', 'TESQ_E_avoidance_of_temptations', 
                          'TESQ_E_goal_deliberation', 'TESQ_E_controlling_temptations', 'TESQ_E_distraction',
                          'TESQ_E_goal_and_rule_setting',
                        'EDM','RS','TRSQ','ROC_Crave_Regulate_Minus_Look','SRHI_unhealthy']:
            display(Markdown("#### " + correlate))
            nan_rows = np.isnan(ind_div_combined_3sd[correlate]) | np.isnan(ind_div_combined_3sd[neural_var])
            cor2way_df = ind_div_combined_3sd.loc[nan_rows==False,]
            pearson_result = pearsonr(cor2way_df[neural_var],cor2way_df[correlate])
            display(HTML("r=" + format(pearson_result[0],".2f") +"; p-value=" + format(pearson_result[1],".4f")))
            spearman_result = spearmanr(cor2way_df[neural_var],cor2way_df[correlate])
            display(HTML("rho=" + format(spearman_result[0],".2f") +"; p-value=" + format(spearman_result[1],".4f")))
            cplot = pyplot.scatter(cor2way_df[neural_var],cor2way_df[correlate])
            cplot.axes.set_xlabel(neural_var)
            cplot.axes.ylabel=correlate
            pyplot.show()

### cache teh subject list

In [6]:
def get_subject_count_with_cache(brain_data_filepath):
    if cache_folder is None:
        cache_folder = (ml_data_folderpath + "/SST/discriminability_cache/" + run_desc + "/")
        
    results_filepath = cache_folder + "dtt_results_" + sample_subject + ".pkl"
    
    print(results_filepath)
    
    if os.path.exists(results_filepath) is False:
        subj_discrim_results = get_subject_discriminability(sample_subject,**subject_discrim_args)
        
        if os.path.exists(cache_folder)==False:
            os.mkdir(cache_folder)
        with open(results_filepath, 'wb') as handle:
            pickle.dump(subj_discrim_results,handle)
    else:
        subj_discrim_results=pickle.load(open(results_filepath,'rb'))
        print("pre-loaded.",flush=True)
        
    print(subj_discrim_results,flush=True)
    return(subj_discrim_results)
    

all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = trialtype_resp_trans_func,
    clean=None)

all_subjects['groups']

subj_list = np.unique(all_subjects['groups'])

NameError: name 'brain_data_filepath' is not defined

## pfc only

In [6]:
dataset_name = 'conditions'


brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_84subs_correct_cond.pkl'
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_58subs_correct_cond.pkl'
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_43subs_correct_cond.pkl'

# def decoderConstructor(*args, **kwargs):
#     return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))

In [7]:
pfc_mask = create_mask_from_images(get_pfc_image_filepaths(ml_data_folderpath + "/"),threshold=10)
relevant_mask = pfc_mask


(8, 238955)
2


In [9]:
all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = trialtype_resp_trans_func,
    clean=None)

all_subjects['groups']

subj_list = np.unique(all_subjects['groups'])

del all_subjects
gc.collect()

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 152
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
clean: 16
subjs_to_use: 16


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


9037
9037


0

In [9]:
summary_results = get_all_subjs_discriminability_masked(
    subj_list, "v_1_pfc",ml_data_folderpath=ml_data_folderpath,
    subject_discrim_args = 
    {'train_test_markers_filepath':train_test_markers_filepath, 
     'brain_data_filepath':brain_data_filepath, 
     'resp_trans_func': trialtype_resp_trans_func,
     'mask':pfc_mask})

DEV005

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/v_1_pfc/dtt_results_DEV005.pkl


pre-loaded.

DEV006

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/v_1_pfc/dtt_results_DEV006.pkl


...

correct-go      85
correct-stop    20
Name: trial_type, dtype: int64

setting up decoder...

fitting

evaluating

{'mean_cv_scores': 0.4000000000000001,
 'overfit_accuracy': 0.7047619047619048,
 'overfit_y_pred_vs_obs':              y_obs        y_pred
 8932    correct-go    correct-go
 8933    correct-go    correct-go
 8934    correct-go  correct-stop
 8935    correct-go    correct-go
 8936    correct-go    correct-go
 ...            ...           ...
 9032    correct-go    correct-go
 9033    correct-go  correct-stop
 9034    correct-go  correct-stop
 9035    correct-go    correct-go
 9036  correct-stop  correct-stop
 
 [105 rows x 2 columns]}

## RTFS

For the by-subject analysis we use a separate source file. The primary reason for this is that we need far fewer conditions in this analysis compared to the other one, because we're only picking out CorrectGo trials that follow a (correct or incorrect) Stop trial.

 - So, the file we use is `Brain_Data_betaseries_84subs_poststop_correctgo.pkl`.
 - We also need a response transform function to pick out the correct column. The column to pick out is `post_stop_correct_go_subtype`.
 - specify a version



In [6]:
def pscg_resp_trans_func(X):
    return(X.post_stop_correct_go_subtype)


In [7]:
version_no = "pscg_v_1_pfc"

brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_84subs_poststop_correctgo.pkl'
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_58subs_correct_cond.pkl'
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_43subs_correct_cond.pkl'

# def decoderConstructor(*args, **kwargs):
#     return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))

In [8]:
pfc_mask = create_mask_from_images(get_pfc_image_filepaths(ml_data_folderpath + "/"),threshold=10)
relevant_mask = pfc_mask


(8, 238955)
2


In [9]:
all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = pscg_resp_trans_func,
    clean=None)

all_subjects['groups']

subj_list = np.unique(all_subjects['groups'])

del all_subjects
gc.collect()

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 158
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
clean: 16
subjs_to_use: 16


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


1686
1686


0

In [12]:
get_subject_discriminability(
    sample_subject='DEV019',
    train_test_markers_filepath=train_test_markers_filepath, 
    brain_data_filepath = brain_data_filepath,
    resp_trans_func = trialtype_resp_trans_func,
    mask=pfc_mask)

loading subject DEV019

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 158
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
subjs_to_use: 64
clean: 60
Brain_Data_allsubs: 48


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


1686
1686


correct-go    22
Name: trial_type, dtype: int64

setting up decoder...

In [12]:
summary_results = get_all_subjs_discriminability_masked(
    subj_list, version_no,ml_data_folderpath=ml_data_folderpath,
    subject_discrim_args = 
    {'train_test_markers_filepath':train_test_markers_filepath, 
     'brain_data_filepath':brain_data_filepath, 
     'resp_trans_func': trialtype_resp_trans_func,
     'mask':pfc_mask})

DEV005

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/pscg_v_1_pfc/dtt_results_DEV005.pkl


loading subject DEV005

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 158
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
subjs_to_use: 64
clean: 60
Brain_Data_allsubs: 48


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


...

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


1686
1686


correct-go    18
Name: trial_type, dtype: int64

setting up decoder...

ValueError: No objects to concatenate

In [1]:
summary_results

NameError: name 'summary_results' is not defined

## trying to figure out why we get different results for overfit discriminability when we do more CV...

Test progressively more granular CV on `TESQ_E_distraction`, `negative emotions_association-test_z_FDR_0`, just a single subject is fine.

In [7]:
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_84subs_correct_cond.pkl'

In [8]:
mask_filepath='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/masks/negative emotions_association-test_z_FDR_0.01.nii.gz'
mask_img = create_mask_from_images([mask_filepath],threshold=0)

In [9]:
get_subject_discriminability(
    sample_subject='DEV019',
    train_test_markers_filepath=train_test_markers_filepath, 
    brain_data_filepath = brain_data_filepath,
    resp_trans_func = trialtype_resp_trans_func,
    mask=mask_img)

loading subject DEV019

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 152
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
subjs_to_use: 64
clean: 60
Brain_Data_allsubs: 48


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


9037
9037
correct-go      96
correct-stop    17
Name: trial_type, dtype: int64
setting up decoder...
using StratifiedKFold with 3 splits


fitting

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/_utils/param_validation.py:197: UserWarning: Brain mask is smaller than .5% of the volume human brain. This object is probably not tuned tobe used on such data.
  screening_percentile_ = _adjust_screening_percentile(


evaluating

{'mean_cv_scores': 0.5388809862494073,
 'overfit_accuracy': 0.8141592920353983,
 'overfit_y_pred_vs_obs':              y_obs        y_pred
 1235    correct-go    correct-go
 1236    correct-go  correct-stop
 1237    correct-go  correct-stop
 1238    correct-go    correct-go
 1239    correct-go    correct-go
 ...            ...           ...
 1343    correct-go    correct-go
 1344    correct-go    correct-go
 1345  correct-stop  correct-stop
 1346    correct-go    correct-go
 1347    correct-go    correct-go
 
 [113 rows x 2 columns],
 'cv_splits': 3,
 'n_samples': 113}

{'mean_cv_scores': 0.5388809862494073,
 'overfit_accuracy': 0.8141592920353983,
 'overfit_y_pred_vs_obs':              y_obs        y_pred
 1235    correct-go    correct-go
 1236    correct-go  correct-stop
 1237    correct-go  correct-stop
 1238    correct-go    correct-go
 1239    correct-go    correct-go
 ...            ...           ...
 1343    correct-go    correct-go
 1344    correct-go    correct-go
 1345  correct-stop  correct-stop
 1346    correct-go    correct-go
 1347    correct-go    correct-go
 
 [113 rows x 2 columns],
 'cv_splits': 3,
 'n_samples': 113}

In [10]:
subj_discrim_results = get_subject_discriminability_with_cache(
    'DEV019',
    'v_1_negative emotions_association-test_z_FDR_0',
    subject_discrim_args = 
    {'train_test_markers_filepath':train_test_markers_filepath, 
     'brain_data_filepath':brain_data_filepath, 
     'resp_trans_func': trialtype_resp_trans_func,
     'mask':mask_img,
     'splits_to_use': "min_class"
    },
    ml_data_folderpath = ml_data_folderpath
)


/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/v_1_negative emotions_association-test_z_FDR_0/dtt_results_DEV019.pkl
pre-loaded.
{'mean_cv_scores': 0.5308202939781888, 'overfit_accuracy': 0.8584070796460177, 'overfit_y_pred_vs_obs':              y_obs        y_pred
1235    correct-go    correct-go
1236    correct-go  correct-stop
1237    correct-go    correct-go
1238    correct-go    correct-go
1239    correct-go    correct-go
...            ...           ...
1343    correct-go  correct-stop
1344    correct-go    correct-go
1345  correct-stop  correct-stop
1346    correct-go    correct-go
1347    correct-go    correct-go

[113 rows x 2 columns]}


In [11]:
results_out = {}
for cv in [3,4,5,6,7,8,11,14,17,18]:
    subj_discrim_results = get_subject_discriminability_with_cache(
        'DEV019',
        'cv_test_' + str(cv) + '_negative emotions_association-test_z_FDR_0',
        subject_discrim_args = 
        {'train_test_markers_filepath':train_test_markers_filepath, 
         'brain_data_filepath':brain_data_filepath, 
         'resp_trans_func': trialtype_resp_trans_func,
         'mask':mask_img,
         'splits_to_use': cv
        },
        ml_data_folderpath = ml_data_folderpath
    )
    results_out[cv] = subj_discrim_results

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/cv_test_3_negative emotions_association-test_z_FDR_0/dtt_results_DEV019.pkl
pre-loaded.
{'mean_cv_scores': 0.4421526789947843, 'overfit_accuracy': 0.8141592920353983, 'overfit_y_pred_vs_obs':              y_obs        y_pred
1235    correct-go    correct-go
1236    correct-go  correct-stop
1237    correct-go    correct-go
1238    correct-go    correct-go
1239    correct-go    correct-go
...            ...           ...
1343    correct-go    correct-go
1344    correct-go  correct-stop
1345  correct-stop  correct-stop
1346    correct-go    correct-go
1347    correct-go    correct-go

[113 rows x 2 columns], 'cv_splits': 3, 'n_samples': 113}
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/cv_test_4_negative emotions_association-test_z_FDR_0/dtt_results_DEV019.pkl
pre-loaded.
{'mean_cv_scores': 0.4605911330049261, 'overfit_accuracy': 0.9026548672566371, 'overfit_y_pred_vs_obs':

loading subject DEV019

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 152
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
subjs_to_use: 64
clean: 60
Brain_Data_allsubs: 48


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


9037
9037
correct-go      96
correct-stop    17
Name: trial_type, dtype: int64
setting up decoder...
using StratifiedKFold with 11 splits


...

fitting

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 17 members, which is less than n_splits=18.
  warnings.warn(("The least populated class in y has only %d"
/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/_utils/param_validation.py:197: UserWarning: Brain mask is smaller than .5% of the volume human brain. This object is probably not tuned tobe used on such data.
  screening_percentile_ = _adjust_screening_percentile(


evaluating

{'mean_cv_scores': 0.4973544973544974,
 'overfit_accuracy': 1.0,
 'overfit_y_pred_vs_obs':              y_obs        y_pred
 1235    correct-go    correct-go
 1236    correct-go    correct-go
 1237    correct-go    correct-go
 1238    correct-go    correct-go
 1239    correct-go    correct-go
 ...            ...           ...
 1343    correct-go    correct-go
 1344    correct-go    correct-go
 1345  correct-stop  correct-stop
 1346    correct-go    correct-go
 1347    correct-go    correct-go
 
 [113 rows x 2 columns],
 'cv_splits': 18,
 'n_samples': 113}

{'mean_cv_scores': 0.4973544973544974, 'overfit_accuracy': 1.0, 'overfit_y_pred_vs_obs':              y_obs        y_pred
1235    correct-go    correct-go
1236    correct-go    correct-go
1237    correct-go    correct-go
1238    correct-go    correct-go
1239    correct-go    correct-go
...            ...           ...
1343    correct-go    correct-go
1344    correct-go    correct-go
1345  correct-stop  correct-stop
1346    correct-go    correct-go
1347    correct-go    correct-go

[113 rows x 2 columns], 'cv_splits': 18, 'n_samples': 113}


In [17]:
[{'cv':x,'overfit_accuracy':results_out[x]['overfit_accuracy']} for x in results_out]

[{'cv': 3, 'overfit_accuracy': 0.8141592920353983},
 {'cv': 4, 'overfit_accuracy': 0.9026548672566371},
 {'cv': 5, 'overfit_accuracy': 0.9557522123893806},
 {'cv': 6, 'overfit_accuracy': 0.9469026548672567},
 {'cv': 7, 'overfit_accuracy': 0.9911504424778761},
 {'cv': 8, 'overfit_accuracy': 0.9911504424778761},
 {'cv': 11, 'overfit_accuracy': 1.0},
 {'cv': 14, 'overfit_accuracy': 1.0},
 {'cv': 17, 'overfit_accuracy': 1.0},
 {'cv': 18, 'overfit_accuracy': 1.0}]

In [18]:
[{
    'cv':x,
    'overfit_accuracy':results_out[x]['overfit_accuracy'],
    'mean_cv_scores':results_out[x]['mean_cv_scores'],
} for x in results_out]

[{'cv': 3,
  'overfit_accuracy': 0.8141592920353983,
  'mean_cv_scores': 0.4421526789947843},
 {'cv': 4,
  'overfit_accuracy': 0.9026548672566371,
  'mean_cv_scores': 0.4605911330049261},
 {'cv': 5,
  'overfit_accuracy': 0.9557522123893806,
  'mean_cv_scores': 0.574703557312253},
 {'cv': 6,
  'overfit_accuracy': 0.9469026548672567,
  'mean_cv_scores': 0.557504873294347},
 {'cv': 7,
  'overfit_accuracy': 0.9911504424778761,
  'mean_cv_scores': 0.539390756302521},
 {'cv': 8,
  'overfit_accuracy': 0.9911504424778761,
  'mean_cv_scores': 0.5208333333333333},
 {'cv': 11, 'overfit_accuracy': 1.0, 'mean_cv_scores': 0.556198347107438},
 {'cv': 14, 'overfit_accuracy': 1.0, 'mean_cv_scores': 0.5843253968253969},
 {'cv': 17, 'overfit_accuracy': 1.0, 'mean_cv_scores': 0.5280112044817927},
 {'cv': 18, 'overfit_accuracy': 1.0, 'mean_cv_scores': 0.4973544973544974}]

`mean_cv_scores` seems to bounce around at random. If we've essentially repeated the experiment with a different random seed and we don't get a value that's robust to multiple correction for 21 repetitions (3 masks and 7 outcome variables) then we probably can't claim a relationship.